In [148]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [149]:
src_path_train = "./Data/train/"
src_path_valid = "./Data/valid/"
src_path_test = "./Data/test/"

scale=1 / 255.0
rotation=20
zoom=0.05
width_shift=0.05
height_shift=0.05
shear=0.05

train_datagen = ImageDataGenerator(
        dtype='float32',
        rescale=scale,
        rotation_range=rotation,
        zoom_range=zoom,
        width_shift_range=width_shift,
        height_shift_range=width_shift,
        shear_range=shear,
        horizontal_flip=True,
        fill_mode="nearest",)

valid_datagen = ImageDataGenerator(
        dtype='float32',
        rescale=scale,
        rotation_range=rotation,
        zoom_range=zoom,
        width_shift_range=width_shift,
        height_shift_range=width_shift,
        shear_range=shear,
        horizontal_flip=True,
        fill_mode="nearest",) 

test_datagen = ImageDataGenerator(
        dtype='float32',
        rescale=scale,
        rotation_range=rotation,
        zoom_range=zoom,
        width_shift_range=width_shift,
        height_shift_range=width_shift,
        shear_range=shear,
        horizontal_flip=True,
        fill_mode="nearest",)

In [171]:
BATCH_SIZE = 32
train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)

valid_generator = valid_datagen.flow_from_directory(
    directory=src_path_valid,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode='categorical',
    shuffle=False,
    seed=42
)

Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.
Found 315 images belonging to 4 classes.


In [151]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization

In [162]:
NUM_CLASSES = 4

CHANNELS = 3

IMAGE_RESIZE = 224
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

LOSS_METRICS = ['accuracy']

NUM_EPOCHS = 50
EARLY_STOP_PATIENCE = 1

BATCH_SIZE_TESTING = 1

STEPS_PER_EPOCH_TRAINING = 20
STEPS_PER_EPOCH_VALIDATION = 1

In [163]:
model = Sequential()

model.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))
model.add(Flatten())
model.add(BatchNormalization())

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
# model.add(Dense(512, activation = DENSE_LAYER_ACTIVATION))
# model.add(Dense(256, activation = DENSE_LAYER_ACTIVATION))
# model.add(Dense(128, activation = DENSE_LAYER_ACTIVATION))
model.add(Dense(NUM_CLASSES, activation = DENSE_LAYER_ACTIVATION))
# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

In [164]:
model.summary()


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_5 (Flatten)         (None, 2048)              0         
                                                                 
 batch_normalization_5 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                                 
 dense_45 (Dense)            (None, 4)                 8196      
                                                                 
Total params: 23604100 (90.04 MB)
Trainable params: 12292 (48.02 KB)
Non-trainable params: 23591808 (90.00 MB)
_________________________________________________________________


In [165]:
from tensorflow.keras import optimizers

sgd = optimizers.SGD(learning_rate = 0.01, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)

In [166]:
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = './models/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [167]:
fit_history = model.fit(
        train_generator,
        steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
        epochs = NUM_EPOCHS,
        validation_data=valid_generator,
        validation_steps=STEPS_PER_EPOCH_VALIDATION,
        callbacks=[cb_checkpointer]
)

Epoch 1/50
20/20 [==============================] - ETA: 0s - loss: 1.0429 - accuracy: 0.5416

/home/ariqhakim/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 8s 290ms/step - loss: 1.0429 - accuracy: 0.5416 - val_loss: 1.5231 - val_accuracy: 0.1562
Epoch 2/50
20/20 [==============================] - 5s 253ms/step - loss: 0.8954 - accuracy: 0.5840 - val_loss: 1.8087 - val_accuracy: 0.1562
Epoch 3/50
20/20 [==============================] - 5s 254ms/step - loss: 0.8629 - accuracy: 0.6378 - val_loss: 1.5018 - val_accuracy: 0.0938
Epoch 4/50
20/20 [==============================] - 5s 255ms/step - loss: 0.8177 - accuracy: 0.6444 - val_loss: 1.3873 - val_accuracy: 0.3438
Epoch 5/50
20/20 [==============================] - 5s 241ms/step - loss: 0.7606 - accuracy: 0.6688 - val_loss: 1.5619 - val_accuracy: 0.2500
Epoch 6/50
20/20 [==============================] - 5s 240ms/step - loss: 0.7558 - accuracy: 0.6705 - val_loss: 1.4141 - val_accuracy: 0.2188
Epoch 7/50
20/20 [==============================] - 5s 261ms/step - loss: 0.7139 - accuracy: 0.6884 - val_loss: 1.2576 - val_accuracy: 0.3438
Epoch 8/50
20/20 

In [172]:
res = model.evaluate(test_generator)

315/315 [==============================] - 7s 16ms/step - loss: 1.1129 - accuracy: 0.5365
